<목차>
- json.load (Naive 방법)
- JsonLoader
  - JSON file
  - JSON Lines file
- jq 스키마 content_key가 포함된 JSON 파일
- metadata 추출

<요약>
- `json, jsonl은 langchain_community.document_loaders.JSONLoader를 이용하여 읽어 들입니다`.
- `jsonl을 읽을 경우 json_lines=True를 합니다`.
- `jq_schema와 content_key를 혼용해서 json에서 특정 구문만 파싱할 수 있습니다`.

※ JSON Lines 사용 시점:  
- 대용량 데이터를 처리해야 할 때
- 데이터를 한 번에 모두 처리하지 않고 한 행씩 처리해야 할 때
- 텍스트 기반 도구를 사용하여 데이터를 처리해야 할 때
- JSON 형식으로 데이터를 저장하고 싶지만 하나의 큰 JSON 파일 대신 여러 개의 작은 파일로 나누고 싶을 때

출처: https://python.langchain.com/docs/modules/data_connection/document_loaders/json/

## JSON
- JSON(JavaScript Object Notation)은 사람이 읽을 수 있는 텍스트를 사용하여 
- 속성-값 쌍과 배열(또는 기타 직렬화 가능한 값)로 구성된 데이터 객체를 저장하고 전송하는
- 개방형 표준 파일 형식 및 데이터 교환 형식입니다.
- [JSON Lines](https://jsonlines.org/)은 각 라인이 유효한 JSON 값인 파일 형식입니다.
- JSONLoader는 지정된 jq 스키마를 사용하여 JSON 파일을 구문 분석합니다. `jq` python 패키지를 사용합니다. jq 구문에 대한 자세한 문서는 이 매뉴얼을 확인하세요.

#### json.load

In [ ]:
from langchain_community.document_loaders import JSONLoader

import json
from pathlib import Path
from pprint import pprint


file_path='./example_data/facebook_chat.json'
data = json.loads(Path(file_path).read_text())
pprint(data) # 원본 파일은 아래와 같습니다.

    {'image': {'creation_timestamp': 1675549016, 'uri': 'image_of_the_chat.jpg'},
     'is_still_participant': True,
     'joinable_mode': {'link': '', 'mode': 1},
     'magic_words': [],
     'messages': [{'content': 'Bye!',
                   'sender_name': 'User 2',
                   'timestamp_ms': 1675597571851},
                  {'content': 'Oh no worries! Bye',
                   'sender_name': 'User 1',
                   'timestamp_ms': 1675597435669},
                  {'content': 'No Im sorry it was my mistake, the blue one is not '
                              'for sale',
                   'sender_name': 'User 2',
                   'timestamp_ms': 1675596277579},
                  {'content': 'I thought you were selling the blue one!',
                   'sender_name': 'User 1',
                   'timestamp_ms': 1675595140251},
                  {'content': 'Im not interested in this bag. Im interested in the '
                              'blue one!',
                   'sender_name': 'User 1',
                   'timestamp_ms': 1675595109305},
                  {'content': 'Here is $129',
                   'sender_name': 'User 2',
                   'timestamp_ms': 1675595068468},
                  {'photos': [{'creation_timestamp': 1675595059,
                               'uri': 'url_of_some_picture.jpg'}],
                   'sender_name': 'User 2',
                   'timestamp_ms': 1675595060730},
                  {'content': 'Online is at least $100',
                   'sender_name': 'User 2',
                   'timestamp_ms': 1675595045152},
                  {'content': 'How much do you want?',
                   'sender_name': 'User 1',
                   'timestamp_ms': 1675594799696},
                  {'content': 'Goodmorning! $50 is too low.',
                   'sender_name': 'User 2',
                   'timestamp_ms': 1675577876645},
                  {'content': 'Hi! Im interested in your bag. Im offering $50. Let '
                              'me know if you are interested. Thanks!',
                   'sender_name': 'User 1',
                   'timestamp_ms': 1675549022673}],
     'participants': [{'name': 'User 1'}, {'name': 'User 2'}],
     'thread_path': 'inbox/User 1 and User 2 chat',
     'title': 'User 1 and User 2 chat'}

※ JSON Lines 사용 시점:  
- 대용량 데이터를 처리해야 할 때
- 데이터를 한 번에 모두 처리하지 않고 한 행씩 처리해야 할 때
- 텍스트 기반 도구를 사용하여 데이터를 처리해야 할 때
- JSON 형식으로 데이터를 저장하고 싶지만 하나의 큰 JSON 파일 대신 여러 개의 작은 파일로 나누고 싶을 때

#### JSONLoader
- JSON 데이터의 `메시지 키 내 콘텐츠 필드 아래의 값`을 추출하는 데 관심이 있다고 가정해 보겠습니다.
- 이는 아래와 같이 JSONLoader를 통해 쉽게 수행할 수 있습니다.  
`==> 본인 같으면 'messages' 이하 부분만 json을 읽어 처리 했겠지만, jq 파이썬 패키지를 이용하여 아래과 같이 할 수도 있습니다.`

#### JSON file

In [ ]:
#!pip install jq
loader = JSONLoader(
    file_path='./example_data/facebook_chat.json',
    jq_schema='.messages[].content', # 설명: 최상위 root(.)에서 messages 키를 찾아. 그리고 이건 배열[]이야. 여기서 content만 수집해
    text_content=False)

data = loader.load()
pprint(data)

    [Document(page_content='Bye!', metadata={'source': '/Users/avsolatorio/WBG/langchain/docs/modules/indexes/document_loaders/examples/example_data/facebook_chat.json', 'seq_num': 1}),
     Document(page_content='Oh no worries! Bye', metadata={'source': '/Users/avsolatorio/WBG/langchain/docs/modules/indexes/document_loaders/examples/example_data/facebook_chat.json', 'seq_num': 2}),
     Document(page_content='No Im sorry it was my mistake, the blue one is not for sale', metadata={'source': '/Users/avsolatorio/WBG/langchain/docs/modules/indexes/document_loaders/examples/example_data/facebook_chat.json', 'seq_num': 3}),
     Document(page_content='I thought you were selling the blue one!', metadata={'source': '/Users/avsolatorio/WBG/langchain/docs/modules/indexes/document_loaders/examples/example_data/facebook_chat.json', 'seq_num': 4}),
     Document(page_content='Im not interested in this bag. Im interested in the blue one!', metadata={'source': '/Users/avsolatorio/WBG/langchain/docs/modules/indexes/document_loaders/examples/example_data/facebook_chat.json', 'seq_num': 5}),
     Document(page_content='Here is $129', metadata={'source': '/Users/avsolatorio/WBG/langchain/docs/modules/indexes/document_loaders/examples/example_data/facebook_chat.json', 'seq_num': 6}),
     Document(page_content='', metadata={'source': '/Users/avsolatorio/WBG/langchain/docs/modules/indexes/document_loaders/examples/example_data/facebook_chat.json', 'seq_num': 7}),
     Document(page_content='Online is at least $100', metadata={'source': '/Users/avsolatorio/WBG/langchain/docs/modules/indexes/document_loaders/examples/example_data/facebook_chat.json', 'seq_num': 8}),
     Document(page_content='How much do you want?', metadata={'source': '/Users/avsolatorio/WBG/langchain/docs/modules/indexes/document_loaders/examples/example_data/facebook_chat.json', 'seq_num': 9}),
     Document(page_content='Goodmorning! $50 is too low.', metadata={'source': '/Users/avsolatorio/WBG/langchain/docs/modules/indexes/document_loaders/examples/example_data/facebook_chat.json', 'seq_num': 10}),
     Document(page_content='Hi! Im interested in your bag. Im offering $50. Let me know if you are interested. Thanks!', metadata={'source': '/Users/avsolatorio/WBG/langchain/docs/modules/indexes/document_loaders/examples/example_data/facebook_chat.json', 'seq_num': 11})]

#### JSON Lines file
- JSON Lines 파일에서 문서를 로드하려면 json_lines=True를 전달하고
- jq_schema를 지정하여 단일 JSON 개체에서 page_content를 추출합니다.

In [ ]:
file_path = './example_data/facebook_chat_messages.jsonl'
pprint(Path(file_path).read_text())

    (
    '{"sender_name": "User 2", "timestamp_ms": 1675597571851, "content": "Bye!"}\n'
     '{"sender_name": "User 1", "timestamp_ms": 1675597435669, "content": "Oh no '
     'worries! Bye"}\n'
     '{"sender_name": "User 2", "timestamp_ms": 1675596277579, "content": "No Im '
     'sorry it was my mistake, the blue one is not for sale"}\n'
     )

In [ ]:
# ==> 아래와 같이 JSON이 아닌 JSON Lines가 있을때 jq 스키마를 이용하는 방법은 다음과 같습니다.
loader = JSONLoader(
    file_path='./example_data/facebook_chat_messages.jsonl',
    jq_schema='.content', # 설명: 최상위 root(.)에서content만 수집해
    text_content=False,
    json_lines=True  # 그리고 이것은 jsonl이야
    )

data = loader.load()

    [Document(page_content='Bye!', metadata={'source': 'langchain/docs/modules/indexes/document_loaders/examples/example_data/facebook_chat_messages.jsonl', 'seq_num': 1}),
     Document(page_content='Oh no worries! Bye', metadata={'source': 'langchain/docs/modules/indexes/document_loaders/examples/example_data/facebook_chat_messages.jsonl', 'seq_num': 2}),
     Document(page_content='No Im sorry it was my mistake, the blue one is not for sale', metadata={'source': 'langchain/docs/modules/indexes/document_loaders/examples/example_data/facebook_chat_messages.jsonl', 'seq_num': 3})]

- 또 다른 옵션은 jq_schema='.'로 설정됩니다. content_key를 제공하세요.

In [ ]:
# jq_schema를 단순히 '.'을 하고, 대신 content_key를 제공하여 파싱 할 수 있습니다.
loader = JSONLoader(
    file_path='./example_data/facebook_chat_messages.jsonl',
    jq_schema='.',  # 설명: 최상위 root(.)에서
    content_key='sender_name', # sender_name만 수집해
    json_lines=True)

data = loader.load()
pprint(data)

    [Document(page_content='User 2', metadata={'source': 'langchain/docs/modules/indexes/document_loaders/examples/example_data/facebook_chat_messages.jsonl', 'seq_num': 1}),
     Document(page_content='User 1', metadata={'source': 'langchain/docs/modules/indexes/document_loaders/examples/example_data/facebook_chat_messages.jsonl', 'seq_num': 2}),
     Document(page_content='User 2', metadata={'source': 'langchain/docs/modules/indexes/document_loaders/examples/example_data/facebook_chat_messages.jsonl', 'seq_num': 3})]

#### jq 스키마 content_key가 포함된 JSON 파일
- jq 스키마 내의 content_key를 사용하여 JSON 파일에서 문서를 로드하려면 is_content_key_jq_parsable=True를 설정합니다. 
- content_key가 호환되고 jq 스키마를 사용하여 구문 분석될 수 있는지 확인하세요.

In [ ]:
file_path = './sample.json'
pprint(Path(file_path).read_text())

    {
    "data": 
        [
        {"attributes": {
            "message": "message1",
            "tags": [
            "tag1"]},
        "id": "1"},
        {"attributes": {
            "message": "message2",
            "tags": [
            "tag2"]},
        "id": "2"}
        ]
    }

In [ ]:
# jq_schema와 content_key를 혼용해서 json에서 파싱할 수 있습니다.
loader = JSONLoader(
    file_path=file_path,
    jq_schema=".data[]",  # 설명: 최상위 root(.)에서
    content_key=".attributes.message",
    is_content_key_jq_parsable=True,
)

data = loader.load()

    [Document(page_content='message1', metadata={'source': '/path/to/sample.json', 'seq_num': 1}),
     Document(page_content='message2', metadata={'source': '/path/to/sample.json', 'seq_num': 2})]

#### metadata 추출
- 일반적으로 우리는 JSON 파일에서 사용할 수 있는 메타데이터를 콘텐츠에서 생성하는 문서에 포함하려고 합니다.
- 다음은 JSONLoader를 사용하여 메타데이터를 추출하는 방법을 보여줍니다.
- 주목해야 할 몇 가지 주요 변경 사항이 있습니다. 메타데이터를 수집하지 않은 이전 예에서는 page_content 값을 추출할 수 있는 스키마에서 직접 지정했습니다.

In [ ]:
.messages[].content

- 현재 예에서는 메시지 필드의 레코드를 반복하도록 로더에 지시해야 합니다. 그러면 jq_schema는 다음과 같아야 합니다.

In [ ]:
.messages[]

- 이를 통해 구현해야 하는 `Metadata_func`에 레코드(dict)를 전달할 수 있습니다.
- `Metadata_func`는 최종 Document 개체에 저장된 메타데이터에 포함되어야 하는 레코드의 정보 부분을 식별하는 역할을 합니다.
- 또한 이제 로더에서 `content_key` 인수를 통해 `page_content` 값을 추출해야 하는 레코드의 키를 명시적으로 지정해야 합니다.

In [ ]:
def metadata_func(record: dict, metadata: dict) -> dict:
    metadata["sender_name"] = record.get("sender_name")
    metadata["timestamp_ms"] = record.get("timestamp_ms")
    return metadata

loader = JSONLoader(
    file_path='./example_data/facebook_chat.json',
    jq_schema='.messages[]',
    content_key="content",
    metadata_func=metadata_func # 메타데이터에 "sender_name"과 "timestamp_ms"를 추가할 수 있습니다.
)

data = loader.load()
pprint(data)

    [Document(page_content='Bye!', metadata={'source': '/Users/avsolatorio/WBG/langchain/docs/modules/indexes/document_loaders/examples/example_data/facebook_chat.json', 'seq_num': 1, 'sender_name': 'User 2', 'timestamp_ms': 1675597571851}),
     Document(page_content='Oh no worries! Bye', metadata={'source': '/Users/avsolatorio/WBG/langchain/docs/modules/indexes/document_loaders/examples/example_data/facebook_chat.json', 'seq_num': 2, 'sender_name': 'User 1', 'timestamp_ms': 1675597435669}),
     Document(page_content='No Im sorry it was my mistake, the blue one is not for sale', metadata={'source': '/Users/avsolatorio/WBG/langchain/docs/modules/indexes/document_loaders/examples/example_data/facebook_chat.json', 'seq_num': 3, 'sender_name': 'User 2', 'timestamp_ms': 1675596277579}),
     Document(page_content='I thought you were selling the blue one!', metadata={'source': '/Users/avsolatorio/WBG/langchain/docs/modules/indexes/document_loaders/examples/example_data/facebook_chat.json', 'seq_num': 4, 'sender_name': 'User 1', 'timestamp_ms': 1675595140251}),
     Document(page_content='Im not interested in this bag. Im interested in the blue one!', metadata={'source': '/Users/avsolatorio/WBG/langchain/docs/modules/indexes/document_loaders/examples/example_data/facebook_chat.json', 'seq_num': 5, 'sender_name': 'User 1', 'timestamp_ms': 1675595109305}),
     Document(page_content='Here is $129', metadata={'source': '/Users/avsolatorio/WBG/langchain/docs/modules/indexes/document_loaders/examples/example_data/facebook_chat.json', 'seq_num': 6, 'sender_name': 'User 2', 'timestamp_ms': 1675595068468}),
     Document(page_content='', metadata={'source': '/Users/avsolatorio/WBG/langchain/docs/modules/indexes/document_loaders/examples/example_data/facebook_chat.json', 'seq_num': 7, 'sender_name': 'User 2', 'timestamp_ms': 1675595060730}),
     Document(page_content='Online is at least $100', metadata={'source': '/Users/avsolatorio/WBG/langchain/docs/modules/indexes/document_loaders/examples/example_data/facebook_chat.json', 'seq_num': 8, 'sender_name': 'User 2', 'timestamp_ms': 1675595045152}),
     Document(page_content='How much do you want?', metadata={'source': '/Users/avsolatorio/WBG/langchain/docs/modules/indexes/document_loaders/examples/example_data/facebook_chat.json', 'seq_num': 9, 'sender_name': 'User 1', 'timestamp_ms': 1675594799696}),
     Document(page_content='Goodmorning! $50 is too low.', metadata={'source': '/Users/avsolatorio/WBG/langchain/docs/modules/indexes/document_loaders/examples/example_data/facebook_chat.json', 'seq_num': 10, 'sender_name': 'User 2', 'timestamp_ms': 1675577876645}),
     Document(page_content='Hi! Im interested in your bag. Im offering $50. Let me know if you are interested. Thanks!', metadata={'source': '/Users/avsolatorio/WBG/langchain/docs/modules/indexes/document_loaders/examples/example_data/facebook_chat.json', 'seq_num': 11, 'sender_name': 'User 1', 'timestamp_ms': 1675549022673})]

#### jq 스키마를 사용한 일반적인 JSON 구조
- 아래 목록은 사용자가 구조에 따라 JSON 데이터에서 콘텐츠를 추출하는 데 사용할 수 있는 가능한 jq_schema에 대한 참조를 제공합니다.


JSON        -> [{"text": ...}, {"text": ...}, {"text": ...}]
jq_schema   -> ".[].text"

JSON        -> {"key": [{"text": ...}, {"text": ...}, {"text": ...}]}
jq_schema   -> ".key[].text"

JSON        -> ["...", "...", "..."]
jq_schema   -> ".[]"

### 원본: './example_data/facebook_chat.json'

    {'image': {'creation_timestamp': 1675549016, 'uri': 'image_of_the_chat.jpg'},
     'is_still_participant': True,
     'joinable_mode': {'link': '', 'mode': 1},
     'magic_words': [],
     'messages': [{'content': 'Bye!',
                   'sender_name': 'User 2',
                   'timestamp_ms': 1675597571851},
                  {'content': 'Oh no worries! Bye',
                   'sender_name': 'User 1',
                   'timestamp_ms': 1675597435669},
                  {'content': 'No Im sorry it was my mistake, the blue one is not '
                              'for sale',
                   'sender_name': 'User 2',
                   'timestamp_ms': 1675596277579},
                  {'content': 'I thought you were selling the blue one!',
                   'sender_name': 'User 1',
                   'timestamp_ms': 1675595140251},
                  {'content': 'Im not interested in this bag. Im interested in the '
                              'blue one!',
                   'sender_name': 'User 1',
                   'timestamp_ms': 1675595109305},
                  {'content': 'Here is $129',
                   'sender_name': 'User 2',
                   'timestamp_ms': 1675595068468},
                  {'photos': [{'creation_timestamp': 1675595059,
                               'uri': 'url_of_some_picture.jpg'}],
                   'sender_name': 'User 2',
                   'timestamp_ms': 1675595060730},
                  {'content': 'Online is at least $100',
                   'sender_name': 'User 2',
                   'timestamp_ms': 1675595045152},
                  {'content': 'How much do you want?',
                   'sender_name': 'User 1',
                   'timestamp_ms': 1675594799696},
                  {'content': 'Goodmorning! $50 is too low.',
                   'sender_name': 'User 2',
                   'timestamp_ms': 1675577876645},
                  {'content': 'Hi! Im interested in your bag. Im offering $50. Let '
                              'me know if you are interested. Thanks!',
                   'sender_name': 'User 1',
                   'timestamp_ms': 1675549022673}],
     'participants': [{'name': 'User 1'}, {'name': 'User 2'}],
     'thread_path': 'inbox/User 1 and User 2 chat',
     'title': 'User 1 and User 2 chat'}